In [ ]:
raw_suicide <- read.csv(file = "SDGSUICIDE.csv", stringsAsFactors = F)

library(ggplot2)
library(dplyr)
suicide <- raw_suicide

#wide pivoting 가능!
# 1. data 추출
suicide_b <- suicide %>% 
  filter(Sex == " Both sexes") %>% 
  select(Country, X20.24.years)
suicide_b <- rename(suicide_b, both = X20.24.years)

suicide_m <- suicide %>% 
  filter(Sex == " Male") %>% 
  select(Country, X20.24.years)
suicide_m <- rename(suicide_m, male = X20.24.years)

suicide_f <- suicide %>% 
  filter(Sex == " Female") %>% 
  select(Country, X20.24.years)
suicide_f <- rename(suicide_f, female = X20.24.years)

# 2. data 합치기
suicide_tot <- left_join(suicide_b, suicide_m, by = "Country")
suicide_tot1 <- left_join(suicide_tot, suicide_f, by = "Country")

# 3. data 수정
suicide_tot2 <- suicide_tot1 %>% 
  filter(both >= 13.5) %>% 
  mutate(differ = male / female) %>% 
  arrange(desc(differ)) %>% 
  head(7)

# 4. 그래프
ggplot(data = suicide_tot2, aes(x = reorder(Country, differ), y = differ))+
  geom_col()+
  coord_flip()+
  ggtitle("20-24 years old Suicide Sex Ratio, Top 7 Countries \n Analyzed by Kim")+
  xlab("Country")+
  ylab("Suicide sex ratio")

In [ ]:
## 1-2. 

install.packages("tidyr") # pivot
library(tidyr)
raw_continent_suicide <- read.csv(file = "countryContinent.csv", stringsAsFactors = F)
continent_suicide <- raw_continent_suicide

# 1. data select
continent <- continent_suicide %>% 
  select(country, continent)
# 2. data 합치기& 정렬, filter
suicide_continent <- left_join(continent, suicide_tot1, by = "country")
suicide_continent <- suicide_continent[ ,c(2, 1, 3, 4, 5)]
suicide_continent2 <- suicide_continent %>% filter(!is.na(both)& !is.na(male)& !is.na(female))

# 3.data pivoting longer, 그룹화, 평균
suicide_continent3 <- suicide_continent2 %>% 
  pivot_longer(c("both", "male", "female"), names_to = "Sex", values_to = "suicide_ratio") 
suicide_continent4 <- suicide_continent3 %>% 
  group_by(continent, Sex) %>%
  summarise(mean_suicide = mean(suicide_ratio))

# 4. 그래프 
ggplot(data = suicide_continent4, aes(x = continent, y = mean_suicide, fill = Sex))+
  geom_col(position = "dodge")+
  scale_x_discrete(limits = c("Africa", "Asia", "Europe", "Americas", "Oceania"))+
  ggtitle("Age 20-24 Suicide Rate By Continent \n Analyzed by Kim")+
  xlab("Continent")+
  ylab("Crude suicide rates(per 100 000 population")

In [ ]:
## 1-3

raw_HR <- read.csv(file = "Human_resources.csv", stringsAsFactors = F)
HR <- raw_HR

#1. rename
HR1 <- rename(HR, psy = Psychiatrists.working.in.mental.health.sector..per.100.000.population.,
              nur =  Nurses.working.in.mental.health.sector..per.100.000.population.,
              soc =  Social.workers.working.in.mental.health.sector..per.100.000.population.,
              psycho = Psychologists.working.in.mental.health.sector..per.100.000.population.)

# 2. NA 제거
HR1$psy <- ifelse(is.na(HR1$psy), 0, HR1$psy)  
HR1$nur <- ifelse(is.na(HR1$nur), 0, HR1$nur) 
HR1$soc <- ifelse(is.na(HR1$soc), 0, HR1$soc)
HR1$psycho <- ifelse(is.na(HR1$psycho), 0, HR1$psycho) 

# 3. HR 합치기, select
HR_tot <- HR1 %>% 
  mutate(total = psy+nur+soc+psycho)
HR_tot1 <- HR_tot %>% 
  select(Country, total)

# 4. suicide_HR(mean, 합치기)
suicide_HR <- suicide %>% 
  filter(Sex == " Both sexes") %>% 
  mutate(mean_suicide = (X15.19.years+X10.14.years+X25.29.years+X20.24.years)/4) %>% 
  select(Country, mean_suicide)
suicide_HR <- left_join(HR_tot1, suicide_HR, by = "Country")

# 5. 그래프
ggplot(data = suicide_HR, aes(x = total, y=mean_suicide ))+
  geom_point()+
  ggtitle("Is there any correlation? \n Hmm..")+
  xlab("HR working in mental health sector(per100 000 popylation)")+
  ylab("10-29 year old suicide rate per 100 000")

In [ ]:
## 2 HR 인구와 남녀 차이와의 관계

#1. 남녀 차이 평균
suicide_differ <- suicide %>% 
  filter(Sex == " Male" | Sex == " Female") %>% 
  mutate(mean_suicide = (X15.19.years+X10.14.years+X25.29.years+X20.24.years)/4) %>% 
  select(Country, Sex, mean_suicide)

# 2. pivot_wider
suicide_differ2 <- suicide_differ %>% 
  pivot_wider(names_from = Sex, values_from = mean_suicide) 
names(suicide_differ2)<- c("Country","Male", "Female") 

# 4. 남녀 차이, 합치기
suicide_differ3 <- suicide_differ2 %>% 
  mutate(differ = abs(Male - Female)) %>% 
  select(Country, differ)
suicide_differ_HR <-left_join(HR_tot1, suicide_differ3, by = "Country")

# 5. 그래프
ggplot(data = suicide_differ_HR, aes(x = total, y=differ ))+
  geom_point()+
  ggtitle("HR working & difference of sex")+
  xlab("HR working in mental health sector(per100 000 popylation)")+
  ylab("difference of 10-29 year old suicide rate ")